**Dependencies:**

- To direct c++ std stream to Jupyter notebook:
    `pip install wurlitzer`

- Deformed frame visualization
    Install `meshcat-python`: https://github.com/rdeits/meshcat-python

In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

In [3]:
import numpy as np
import pyconmech as cm
from wurlitzer import sys_pipes

import meshcat

import meshcat.geometry as g
from deformed_frame_viz import meshcat_visualize_deformed

In [4]:
vis = meshcat.Visualizer()
vis.jupyter_cell()

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7000/static/


# check deformation for a completed structure

In [8]:
import os
import time
from deformed_frame_viz import meshcat_visualize_deformed
cwd = os.getcwd()

json_path = os.path.join(cwd, "assembly_instances/extrusion","topopt-100_S1_03-14-2019_w_layer.json")
# json_path = os.path.join(cwd,"sf-test_3-frame.json")
# json_path = os.path.join(cwd,"tower_3D.json")
# load_case_path = os.path.join(cwd,"tower_3D_load_case.json")

vis.delete()
disc = 10
exagg_ratio = 1.0
time_step = 1.0
scale = 5

with sys_pipes():
    sc = cm.stiffness_checker(json_file_path = json_path, verbose = True)
    sc.set_output_json(True)
    sc.set_output_json_path(file_path = cwd, file_name = "sf-test_result.json")

    # manually assigned point loads
#     ext_load = np.zeros([1,7])
#     ext_load[0,0] = 3
#     ext_load[0,4] = -1
    include_sw = True

    # import load from a json file
#     ext_load, include_sw = cm.parse_load_case_from_json(load_case_path)
#     print('external load:')
#     print(ext_load)
#     print('include self weight : {0}'.format(include_sw))
#     sc.set_load(nodal_forces = ext_load)
    sc.set_self_weight_load(include_sw) # now this is true by default
    
#     sc.set_nodal_displacement_tol(transl_tol=1e-3)
    
#     existing_ids = [125, 126, 115, 122, 111, 108, 23, 22, 98, 75, 64, 34, 61, 65, 59, 60, 39, 36, 44, 67]
    existing_ids = [124, 126, 127, 125, 53, 48, 71, 58, 57, 65, 63, \
                    21, 69, 64, 82, 70, 18, 91, 85, 47, 84, 78, 89, 108, 86, 98, 33]#, 36, 22, 90, 87]
#     existing_ids = [64] # unsupported
    
    print("solve: {0}\n\n".format(sc.solve(existing_ids)))
    
    # Collecting results
#     print("has stored results?: {0}".format(sc.has_stored_result()))    
#     success, nD, fR, eR = sc.get_solved_results()
#     print("pass criteria?\n {0}".format(success))
#     print("nodal displacement?\n {0}".format(nD))
#     print("fixities reaction?\n {0}".format(fR))
#     print("element reaction?\n {0}".format(eR))
    
    # visualize deformed structure
    orig_beam_shape = sc.get_original_shape(disc=disc, draw_full_shape=False)
#     print(orig_beam_shape)
    
    beam_disp = sc.get_deformed_shape(exagg_ratio=exagg_ratio, disc=disc)
#     print(beam_disp)
    meshcat_visualize_deformed(vis, beam_disp, orig_beam_shape, disc=disc, scale=scale)

solve: False


Node #68: No fixities info specified, assuming to be 6-dof fixed.
Node #69: No fixities info specified, assuming to be 6-dof fixed.
Node #70: No fixities info specified, assuming to be 6-dof fixed.
Node #71: No fixities info specified, assuming to be 6-dof fixed.
Node #72: No fixities info specified, assuming to be 6-dof fixed.
Node #73: No fixities info specified, assuming to be 6-dof fixed.
Node #74: No fixities info specified, assuming to be 6-dof fixed.
Node #75: No fixities info specified, assuming to be 6-dof fixed.
Node #76: No fixities info specified, assuming to be 6-dof fixed.
Node #77: No fixities info specified, assuming to be 6-dof fixed.
Node #78: No fixities info specified, assuming to be 6-dof fixed.
Node #79: No fixities info specified, assuming to be 6-dof fixed.
Node #80: No fixities info specified, assuming to be 6-dof fixed.
Node #81: No fixities info specified, assuming to be 6-dof fixed.
Node #82: No fixities info specified, assuming to be 6-dof fi

# animate deformation for a construction sequence

In [7]:
import os
import time
cwd = os.getcwd()

json_path = os.path.join(cwd,"tower_3D.json")
# json_path = os.path.join(cwd,"sf-test_3-frame.json")
load_case_path = os.path.join(cwd,"tower_3D_load_case.json")

vis.delete()
disc = 10
exagg_ratio=1.0
time_step = 1.0

with sys_pipes():
    sc = cm.stiffness_checker(json_file_path = json_path, verbose = False)
    
#     sc.set_output_json(True)
#     sc.set_output_json_path(file_path = cwd, file_name = "sf-test_result.json")

#     ext_load = np.zeros([1,7])
#     ext_load[0,0] = 3
#     include_sw = False

#     ext_load, include_sw = cm.parse_load_case_from_json(load_case_path)
#     print('external load:')
#     print(ext_load)
#     print('include self weight : {0}'.format(include_sw))
#     sc.set_load(nodal_forces = ext_load)
#     sc.set_self_weight_load(include_sw)

    sc.set_self_weight_load(True)

    for i in range(0,24):
        vis.delete()
        existing_ids = list(range(0,i+1))
        sc.solve(existing_ids)

        orig_beam_shape = sc.get_original_shape(disc=disc, draw_full_shape=False)
        beam_disp = sc.get_deformed_shape(exagg_ratio=exagg_ratio, disc=disc)
        meshcat_visualize_deformed(vis, beam_disp, orig_beam_shape, disc=disc, scale=0.5)
        time.sleep(time_step)